# EXTERNAL

In [3]:
#@title OpenBioLink Module Install EXTERNAL{ form-width: "15%" }
! pip install openbiolink

     |████████████████████████████████| 226 kB 5.3 MB/s 


In [4]:
#@title PyG 113 Installation EXTERNAL{ form-width: "15%" }
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 2.8 MB/s 
     |████████████████████████████████| 3.5 MB 2.8 MB/s 
     |████████████████████████████████| 482 kB 5.4 MB/s 
     |████████████████████████████████| 41 kB 181 kB/s 


# HEADER

In [6]:
#@title Module Imports HEADER { form-width: "15%" }
from openbiolink.obl2021 import OBL2021Dataset, OBL2021Evaluator
import torch
from torch.nn import Module,\
                     ModuleList,\
                     Embedding,\
                     BatchNorm1d,\
                     Linear,\
                      BCEWithLogitsLoss,\
                     CrossEntropyLoss,\
                     Dropout
from torch.optim import Adam
import torch.nn.functional as F
import torch_geometric as PyG
from torch_geometric.data import Data
from torch_geometric.nn.conv import RGCNConv
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from typing import NoReturn
import typing
import time
from enum import Enum
from datetime import datetime
from collections import defaultdict

# GLOBAL 
<!-- ###now CPU -->

In [7]:
#@title Global Variables CLASS{ form-width: "15%" }
class Global(Enum):
  HEAD_INDEX = 0
  RELATION_INDEX = 1
  TAIL_INDEX = 2
  FEATURE_ENG = 'one-hot'
  NUM_RELATIONS = 28
  DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
  MINI_BATCH_SIZE = 32
  DISP_HITS = False
  MANUAL_SEED = 2021


# torch.manual_seed(Global.MANUAL_SEED.value)
dataset = OBL2021Dataset()
evaluator = OBL2021Evaluator()
kg = torch.cat((dataset.training, dataset.validation, dataset.testing), dim=0)
sorted_on_tails_indcs_kg = torch.sort(kg[:, Global.TAIL_INDEX.value])[1]
kg_sorted_tails = kg[sorted_on_tails_indcs_kg]
sorted_on_tails_indcs_kg = None
sorted_on_heads_indcs_kg = torch.sort(kg[:, Global.HEAD_INDEX.value])[1]
kg_sorted_heads = kg[sorted_on_heads_indcs_kg]
sorted_on_heads_indcs_kg = None
features = dataset.candidates.to(Global.DEVICE.value) 
train_set = dataset.training.to(Global.DEVICE.value) # torch.tensor of shape(num_train,3)
val_set = dataset.validation.to(Global.DEVICE.value) # torch.tensor of shape(num_val,3)
test_set = dataset.testing.to(Global.DEVICE.value)   # torch.tensor of shape(num_train,3)
dataset = None
sorted_train_set = train_set[torch.sort(train_set[:, 0])[1]]

Dataset not found, downloading to /content/obl2021 ...


KGID_HQ_DIR.zip: 45.2MB [01:23, 570kB/s]                            


# GNN Model and Hyperparameters

In [8]:
#@title GNN Model CLASS{ form-width: "10%" }
class GNN(Module):
  def __init__(self, conv_dims: list, fully_connected_for_head_dims: list, fully_connected_for_tail_dims: list, x_feature:str, dropout: dict, embedding_dims: tuple=None)-> NoReturn:
    super(GNN, self).__init__()
    self.conv_dropout = Dropout(p=dropout["conv"])
    self.emb_dropout = Dropout(p=dropout["emb"]) 
    self.fc_dropout = Dropout(p=dropout["fc"])
    self.x_feature = x_feature
    self.num_relations = Global.NUM_RELATIONS.value
    self.relation_embedder = Embedding(self.num_relations, fully_connected_for_tail_dims[-1])
    if x_feature == 'one-hot':
      self.entity_embedder = Embedding(embedding_dims[0], embedding_dims[1])
      first_conv_layer = [RGCNConv(embedding_dims[1], conv_dims[0], self.num_relations)]
    elif x_feature == 'identity':
      first_conv_layer = [RGCNConv(1, conv_dims[0], self.num_relations)]
    conv_list = first_conv_layer + \
                                [
                                  RGCNConv(conv_dims[i], conv_dims[i+1], self.num_relations)
                                  for i in range(len(conv_dims[:-1]))
                                ]
  
    fully_connected_list_for_tail = [
                                      Linear(conv_dims[-1], fully_connected_for_tail_dims[0]) 
                                    ] +\
                                    [
                                     Linear(fully_connected_for_tail_dims[i], fully_connected_for_tail_dims[i + 1])
                                     for i in range(len(fully_connected_for_tail_dims[:-1]))
                                    ]

    fully_connected_list_for_head = [
                                      Linear(conv_dims[-1], fully_connected_for_head_dims[0]) 
                                    ] +\
                                    [
                                     Linear(fully_connected_for_head_dims[i], fully_connected_for_head_dims[i + 1])
                                     for i in range(len(fully_connected_for_head_dims[:-1]))
                                    ]
    

    #graph conv layers
    self.conv_layers = ModuleList(conv_list)

    #fully connected dense layers
    self.head_fc_layers = ModuleList(fully_connected_list_for_head)
    self.tail_fc_layers = ModuleList(fully_connected_list_for_tail)
    

  def reset_parameters(self):
    self.entity_embedder.reset_parameters()
    self.relation_embedder.reset_parameters()
    for conv in self.conv_layers:
        conv.reset_parameters()
    for fc in self.head_fc_layers:
        fc.reset_parameters()
    for fc in self.tail_fc_layers:
        fc.reset_parameters()


  def forward(self, data: Data) -> torch.Tensor:
    edge_index = data.edge_index_messaging
    x = data.x
    edge_type = data.edge_type_messaging

    ####################################### One-Hot Entity Encoder #######################################
    if self.x_feature == 'one-hot':
      x = self.entity_embedder(x).reshape(self.entity_embedder.weight.shape[0], -1)
    if self.training:
        x = self.emb_dropout(x)
    ############################################## Identity Encoder ################################################
    elif self.x_feature == 'identity':
      x = torch.ones(x.shape[0], 1)
      if self.training:
        x = self.emb_dropout(x)
    ####################################### RGCN Encoder #######################################
    # for conv in self.conv_layers[:-1]:
    #   x = conv(x, edge_index=edge_index, edge_type=edge_type)
    #   x = F.relu(x)
    #   if self.training:
    #     x = self.conv_dropout(x)
    # x = self.conv_layers[-1](x, edge_index, edge_type)
    # if self.training:
    #   x = self.conv_dropout(x)
    
    ####################################### Decoder #######################################
    
    positive_heads = x[data.edge_index_supervision[0]]
    positive_tails = x[data.edge_index_supervision[1]]
    positive_relations = self.relation_embedder(data.edge_type_supervision) # relation encoder

    negative_heads = x[data.edge_index_negative[0]]
    negative_tails = x[data.edge_index_negative[1]]
    negative_relations = self.relation_embedder(data.edge_type_negative)

    heads = torch.cat((positive_heads, negative_heads))
    relations = torch.cat((positive_relations, negative_relations))
    tails = torch.cat((positive_tails, negative_tails))
    perm_index = torch.randperm(heads.shape[0])
    heads = heads[perm_index]
    tails = tails[perm_index]
    relations = relations[perm_index]

    if self.training:
      # negative_relations = self.emb_dropout(negative_relations)
      # positive_relations = self.emb_dropout(positive_relations)
      relations = self.emb_dropout(relations)

    # for head_fc, tail_fc in zip(self.head_fc_layers[:-1], self.tail_fc_layers[:-1]):
    #   heads = head_fc(heads)
    #   heads = F.relu(heads)
    #   tails = tail_fc(tails)
    #   tails = F.relu(tails)

    #   if self.training:
    #     heads = self.conv_dropout(heads)
    #     tails = self.conv_dropout(tails)
    
    # heads = self.head_fc_layers[-1](heads)
    # tails = self.tail_fc_layers[-1](tails)
    # if self.training:
    #     heads = self.conv_dropout(heads)
    #     tails = self.conv_dropout(tails)
    
    # let f be the fully connected transform function for heads, and 
    # let g be the fully connected transform function for tails:
        # prior to the fully connected transforms:
        #     score(head, relation, tail) = <head, relation, tail> = head . relation . tail
        #     score(tail, relation, head) = <tail, relation, head> = tail . relation . head
        #     ===> score(head, relation, tail) = score(tail, relation, head) essentially holds
        # now with the fully connected transforms:
        #     score(head, relation, tail) = <f(head), relation, g(tail)> = f(head) . relation . g(tail)
        #     score(tail, relation, head) = <f(tail), relation, g(head)> = f(tail) . relation . g(head)
        #     ===> score(head, relation, tail) = score(tail, relation, head) does not necessarily holds

    scores = (heads * relations * tails).sum(dim=1)
    original_index = torch.sort(perm_index)[1]
    return scores[original_index[:positive_heads.shape[0]]], scores[original_index[positive_heads.shape[0]:]]
    # return (positive_heads * positive_relations * positive_tails).sum(dim=1), \
    #        (negative_heads * negative_relations * negative_tails).sum(dim=1)

In [9]:
#@title Shallow Model Class

class ShallowModel(Module):
  def __init__(self, x_feature:str, dropout: dict, embedding_dims: tuple=None)-> NoReturn:
    super(ShallowModel, self).__init__()
    # self.conv_dropout = Dropout(p=dropout["conv"])
    self.emb_dropout = Dropout(p=dropout["emb"]) 
    # self.fc_dropout = Dropout(p=dropout["fc"])
    self.x_feature = x_feature
    self.num_relations = Global.NUM_RELATIONS.value
    self.relation_embedder = Embedding(self.num_relations, embedding_dims[-1])
    if x_feature == 'one-hot':
      self.entity_embedder = Embedding(embedding_dims[0], embedding_dims[1])
    #   first_conv_layer = [RGCNConv(embedding_dims[1], conv_dims[0], self.num_relations)]
    # elif x_feature == 'identity':
    #   first_conv_layer = [RGCNConv(1, conv_dims[0], self.num_relations)]
    # conv_list = first_conv_layer + \
    #                             [
    #                               RGCNConv(conv_dims[i], conv_dims[i+1], self.num_relations)
    #                               for i in range(len(conv_dims[:-1]))
    #                             ]
  
    # fully_connected_list_for_tail = [
    #                                   Linear(conv_dims[-1], fully_connected_for_tail_dims[0]) 
    #                                 ] +\
    #                                 [
    #                                  Linear(fully_connected_for_tail_dims[i], fully_connected_for_tail_dims[i + 1])
    #                                  for i in range(len(fully_connected_for_tail_dims[:-1]))
    #                                 ]

    # fully_connected_list_for_head = [
    #                                   Linear(conv_dims[-1], fully_connected_for_head_dims[0]) 
    #                                 ] +\
    #                                 [
    #                                  Linear(fully_connected_for_head_dims[i], fully_connected_for_head_dims[i + 1])
    #                                  for i in range(len(fully_connected_for_head_dims[:-1]))
    #                                 ]
    

    #graph conv layers
    # self.conv_layers = ModuleList(conv_list)

    #fully connected dense layers
    # self.head_fc_layers = ModuleList(fully_connected_list_for_head)
    # self.tail_fc_layers = ModuleList(fully_connected_list_for_tail)
    

  def reset_parameters(self):
    self.entity_embedder.reset_parameters()
    self.relation_embedder.reset_parameters()
    # for conv in self.conv_layers:
    #     conv.reset_parameters()
    # for fc in self.head_fc_layers:
    #     fc.reset_parameters()
    # for fc in self.tail_fc_layers:
    #     fc.reset_parameters()


  def forward(self, data: Data) -> torch.Tensor:
    edge_index = data.edge_index_messaging
    x = data.x
    edge_type = data.edge_type_messaging

    ####################################### One-Hot Entity Encoder #######################################
    if self.x_feature == 'one-hot':
      x = self.entity_embedder(x).reshape(self.entity_embedder.weight.shape[0], -1)
    ############################################## Identity Encoder ################################################
    elif self.x_feature == 'identity':
      x = torch.ones(x.shape[0], 1).to(Global.DEVICE.value)
      
    if self.training:
      x = self.emb_dropout(x)
    ####################################### RGCN Encoder #######################################
    # for conv in self.conv_layers[:-1]:
    #   x = conv(x, edge_index=edge_index, edge_type=edge_type)
    #   x = F.relu(x)
    #   if self.training:
    #     x = self.conv_dropout(x)
    # x = self.conv_layers[-1](x, edge_index, edge_type)
    # if self.training:
    #   x = self.conv_dropout(x)
    
    ####################################### Decoder #######################################
    positive_heads = x[data.edge_index_supervision[0]]
    positive_tails = x[data.edge_index_supervision[1]]
    positive_relations = data.edge_type_supervision

    negative_heads = x[data.edge_index_negative[0]]
    negative_tails = x[data.edge_index_negative[1]]
    negative_relations = data.edge_type_negative

    heads = torch.cat((positive_heads, negative_heads))
    relations = torch.cat((positive_relations, negative_relations))
    relations = self.relation_embedder(relations)
    tails = torch.cat((positive_tails, negative_tails))

    perm_index = torch.randperm(heads.shape[0])
    heads = heads[perm_index]
    tails = tails[perm_index]
    relations = relations[perm_index]

    if self.training:
      relations = self.emb_dropout(relations)

    # for head_fc, tail_fc in zip(self.head_fc_layers[:-1], self.tail_fc_layers[:-1]):
    #   heads = head_fc(heads)
    #   heads = F.relu(heads)
    #   tails = tail_fc(tails)
    #   tails = F.relu(tails)

    #   if self.training:
    #     heads = self.conv_dropout(heads)
    #     tails = self.conv_dropout(tails)

    # heads = self.head_fc_layers[-1](heads)
    # tails = self.tail_fc_layers[-1](tails)

    # if self.training:
    #   heads = self.conv_dropout(heads)
    #   tails = self.conv_dropout(tails)
    # let f be the fully connected transform function for heads, and 
    # let g be the fully connected transform function for tails:
        # prior to the fully connected transforms:
        #     score(head, relation, tail) = <head, relation, tail> = head . relation . tail
        #     score(tail, relation, head) = <tail, relation, head> = tail . relation . head
        #     ===> score(head, relation, tail) = score(tail, relation, head) essentially holds
        # now with the fully connected transforms:
        #     score(head, relation, tail) = <f(head), relation, g(tail)> = f(head) . relation . g(tail)
        #     score(tail, relation, head) = <f(tail), relation, g(head)> = f(tail) . relation . g(head)
        #     ===> score(head, relation, tail) = score(tail, relation, head) does not necessarily holds

    scores = (heads * relations * tails).sum(dim=1)
    original_index = torch.sort(perm_index)[1]
    return scores[original_index[:positive_heads.shape[0]]], scores[original_index[positive_heads.shape[0]:]]


In [10]:
#@title Model and Hyperparameters MAIN { form-width: "15%" }
model = GNN(
    x_feature='one-hot',
    conv_dims=[32, 32, 32],
    embedding_dims=(features.max() + 1, 32),
    fully_connected_for_head_dims=[32, 32], 
    fully_connected_for_tail_dims=[32, 32],
    dropout={
        "emb": 0.1,
        "conv": 0.1,
        "fc": 0.3
    }
).to(Global.DEVICE.value)
print(model)
shallow_model = ShallowModel('one-hot', {
        "emb": 0.1,
        "conv": 0.1,
        "fc": 0.3
    }, (features.max() + 1, 32)).to(Global.DEVICE.value)
loss_fn = torch.nn.BCEWithLogitsLoss()
hinge_embedding_loss_fn = torch.nn.HingeEmbeddingLoss()
margin_ranking_loss = torch.nn.MarginRankingLoss(margin=25, reduction='sum')
opt = Adam(model.parameters())
opt_shallow = Adam(shallow_model.parameters())

GNN(
  (conv_dropout): Dropout(p=0.1, inplace=False)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (fc_dropout): Dropout(p=0.3, inplace=False)
  (relation_embedder): Embedding(28, 32)
  (entity_embedder): Embedding(180992, 32)
  (conv_layers): ModuleList(
    (0): RGCNConv(32, 32, num_relations=28)
    (1): RGCNConv(32, 32, num_relations=28)
    (2): RGCNConv(32, 32, num_relations=28)
  )
  (head_fc_layers): ModuleList(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
  )
  (tail_fc_layers): ModuleList(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
  )
)


#FUNCTIONS

In [51]:
#@title find_start_and_end_indcs() PRIVATE-FUNCTION { form-width: "15%" }
def find_start_and_end_indcs(entity, relation, entity_type, kg_sorted):
  up = kg_sorted.shape[0]
  down = 0
  indx = kg_sorted.shape[0] // 2
  found = False
  while up - down > 1:
    if kg_sorted[indx][entity_type].item() == entity:
      found = True
      break 
    elif kg_sorted[indx][entity_type].item() >= entity: 
      up = indx
      indx = (up + down) // 2
    else:
      down = indx 
      indx = (up + down) // 2
  if not found:
    return None
  while 1:
    indx += 1
    try:
      if not kg_sorted[indx][entity_type].item() == entity:
        indx -= 1
        end_indx = indx
        break
    except:
      end_indx = indx - 1
      break
  

  while 1:
    indx -= 1
    try:
      if not kg_sorted[indx][entity_type].item() == entity:
        indx += 1
        start_indx = indx
        break
    except:
      start_indx = indx + 1
      break
  indcs = torch.arange(start_indx, end_indx + 1)
  tns = kg_sorted[indcs]
  srt_indcs = torch.sort(tns[:, 1])[1]

  msk = tns[srt_indcs][:, 1] == relation
  okay_indcs = torch.nonzero(msk).reshape(-1)
  return indcs[srt_indcs[okay_indcs]]

In [53]:
#@title get_psuedo_negative_entities() FUNCTIONS { form-width: "15%" }
def get_psuedo_negative_entities(entity, relation, corrupt_at, kg_sorted):
  if corrupt_at == Global.HEAD_INDEX.value:
    entity_type = Global.TAIL_INDEX.value 
  elif corrupt_at == Global.TAIL_INDEX.value:
    entity_type = Global.HEAD_INDEX.value

  indcs = find_start_and_end_indcs(entity, relation, entity_type, kg_sorted=kg_sorted)
  
  if indcs is not None:
    fact_triplets_entities = kg_sorted[indcs][:, corrupt_at]
    features_copy = features.detach().clone()
    features_copy[fact_triplets_entities] = -1
    non_negative_mask = features_copy >= 0
    ret = torch.nonzero(non_negative_mask).reshape(-1)
    return ret
  else:
    return features.to(Global.DEVICE.value)

In [13]:
#@title check_negative_samples() FUNCTION { form-width: "15%" }
def check_negative_samples(negative_samples, sorted_train_set):
  for triple in negative_samples:
    indcs = find_start_and_end_indcs(triple[0], 0, sorted_train_set)
    if isinstance(indcs, torch.Tensor):
      facts = sorted_train_set[indcs]
      are_actually_corrupt = ((triple == facts).type(torch.int8).sum(dim=-1) < 3).sum()
      if not are_actually_corrupt.type(torch.int8):
        return False 
  return True

In [14]:
#@title psuedo negative triplets { form-width: "15%" }
def get_psuedo_negative_triplets(triplet: torch.Tensor) -> tuple:
  head = triplet[Global.HEAD_INDEX.value].item()
  relation = triplet[Global.RELATION_INDEX.value].item()
  tail = triplet[Global.TAIL_INDEX.value].item()

  corrupted_tails = get_psuedo_negative_entities(
      entity=head, 
      relation=relation,
      corrupt_at=Global.TAIL_INDEX.value, 
      kg_sorted=kg_sorted_heads
    )
  
  corrupted_heads = get_psuedo_negative_entities(
        entity=tail, 
        relation=relation,
        corrupt_at=Global.HEAD_INDEX.value, 
        kg_sorted=kg_sorted_tails
    )

  
  num_psuedo_negative_triples = corrupted_tails.shape[0]
  psuedo_negative_triplets_corrupted_tail = torch.vstack(
      (
          torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * head,
          torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * relation,
          corrupted_tails
      )
  ).t().to(Global.DEVICE.value)

  num_psuedo_negative_triples = corrupted_heads.shape[0]
  psuedo_negative_triplets_corrupted_head = torch.vstack(
      (
          corrupted_heads,
          torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * relation,
          torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * tail
      )
  ).t().to(Global.DEVICE.value)
  
  
  return psuedo_negative_triplets_corrupted_head , psuedo_negative_triplets_corrupted_tail
  

In [15]:

#@title #TODO mini_batch_maker NEGATIVE SAMPLING use MSCaching FUNCTION  { form-width: "15%" }
def mini_batch_maker(messaging, supervision, candidates, x_feature='one-hot'):
  heads = supervision[:, 0]
  relations = supervision[:, 1]
  tails = supervision[:, -1]

  ct_size = supervision.shape[0] // 2
  ch_size = supervision.shape[0] - ct_size
  while 1:
    negative_samples_corrupted_tails = torch.vstack(
        (
            heads[: ct_size], 
            relations[: ct_size], 
            torch.multinomial(
                candidates.type(torch.float).to(Global.DEVICE.value), 
                ct_size
            )
        )
    ).t().contiguous()

    negative_samples_corrupted_heads = torch.vstack(
        ( 
            torch.multinomial(
                candidates.type(torch.float).to(Global.DEVICE.value), 
                ch_size
            ), 
            relations[ch_size:], 
            tails[ch_size:]
        )
    ).t().contiguous()

    negative_samples = torch.cat(
        (negative_samples_corrupted_heads, negative_samples_corrupted_tails),
        dim=0
    )
    break
    if check_negative_samples(negative_samples, sorted_train_set):
      break


  relations = supervision[:, 1]
  while True:
    batch_size = supervision.shape[0]
    negative_samples = torch.vstack(
        (
            torch.multinomial(candidates.type(torch.float).to(Global.DEVICE.value), batch_size),
            relations,
            torch.multinomial(candidates.type(torch.float).to(Global.DEVICE.value), batch_size)
        )
    ).t().contiguous()
    if check_negative_samples(negative_samples, sorted_train_set):
      break

  graph = graph_data_maker(
      messaging=messaging,
      supervision=supervision,
      negative_samples=negative_samples,
      x=candidates.to(Global.DEVICE.value),
      x_feature='one-hot'
  )

  return graph

In [57]:
#@title graph_data_maker(dataset, x_feature) FUNCTION{ form-width: "15%" }
def graph_data_maker(messaging: torch.Tensor, supervision: torch.Tensor, negative_samples: torch.Tensor, x:torch.Tensor, x_feature: str=Global.FEATURE_ENG.value, check_for_correctness: bool=False) -> Data:
  relation_idx = Global.RELATION_INDEX.value
  head_idx = Global.HEAD_INDEX.value
  tail_idx = Global.TAIL_INDEX.value
  graph_data = Data(
        x=x.reshape(-1, 1),
        edge_index_messaging=messaging[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_messaging=messaging[:, relation_idx],
        edge_index_supervision=supervision[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_supervision=supervision[:, relation_idx],
        edge_index_negative=negative_samples[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_negative=negative_samples[:, (relation_idx)]
    )
  return graph_data


In [58]:
#@title train(model, graph, optimizer, loss_fn) FUNCTION{ form-width: "15%" }
def train(model: GNN, graph: Data, optimizer: torch.optim, loss_fn:torch.nn.modules.loss) -> tuple((torch.float, torch.Tensor, torch.Tensor)):
  optimizer.zero_grad()
  model.train()
  positive, negative = model(graph)
  scores = torch.cat((positive, negative))
  labels = torch.ones(graph.edge_index_supervision.shape[1]).to(Global.DEVICE.value)
  loss = loss_fn(positive, negative, labels)
  # targets = torch.cat(
  #     (
  #         torch.ones(graph.edge_index_supervision.shape[1]).to(Global.DEVICE.value),
  #         torch.ones(graph.edge_index_negative.shape[1]).to(Global.DEVICE.value) * -1
  #      )
  # )
  # # loss = loss_fn(
  # #           negative, 
  # #           torch.zeros(graph.edge_index_negative.shape[1]).to(Global.DEVICE.value)
  # #        ) + loss_fn(
  # #               positive,
  # #               torch.ones(graph.edge_index_supervision.shape[1]).to(Global.DEVICE.value)
  # #           )
  # loss = loss_fn(scores, targets)
  loss.backward()
  optimizer.step()
  return loss.item(), positive, negative

In [59]:
#@title evaluation_rank() PRIVATE-FUNCTION { form-width: "10%" }
@torch.no_grad()
def evaluation_rank(model, eval_set, messaging_set):
  h_pred_top10 = list()
  t_pred_top10 = list()

  for eval_triplet in tqdm(eval_set):
    #use tqdm
    psuedo_negative_triplets_corrupted_head, psuedo_negative_triplets_corrupted_tail = get_psuedo_negative_triplets(eval_triplet)


    # eval_triplet: (h, r, t)
    # psuedo_negative_triplets_head: (h′, r, t) for all h′
    # psuedo_negative_triplets_tail: (h, r, t′) for all t′

    # train_set being the messaging graph, calculate the score for (h, r, t)
    # train_set being the messaging graph, calculate the scores for all (h′, r, t)
    # train_set being the messaging graph, calculate the scores for all (h, r, t′)

    graph_data_for_object_tail = graph_data_maker(
      messaging=messaging_set,
      supervision=eval_triplet.reshape(1, 3),
      negative_samples=psuedo_negative_triplets_corrupted_tail,
      x=features.to(Global.DEVICE.value),
      x_feature='one-hot'
    )

    graph_data_for_object_head = graph_data_maker(
            messaging=messaging_set,
            supervision=eval_triplet.reshape(1, 3),
            negative_samples=psuedo_negative_triplets_corrupted_head,
            x=features.to(Global.DEVICE.value),
            x_feature='one-hot'
    )

    model.eval()
    scores_for_object_tail = torch.cat(model(graph_data_for_object_tail))

    sorted_by_scores_for_object_tail_indcs = torch.sort(scores_for_object_tail, descending=True)[1]

    tail_objects = torch.cat((graph_data_for_object_tail.edge_index_supervision[1], graph_data_for_object_tail.edge_index_negative[1]))
    top10_tails = tail_objects[sorted_by_scores_for_object_tail_indcs[:10]]
    t_pred_top10.append(top10_heads)

    model.eval()
    scores_for_object_head = torch.cat(model(graph_data_for_object_head))

    sorted_by_scores_for_object_head_indcs = torch.sort(scores_for_object_head, descending=True)[1]

    head_objects = torch.cat((graph_data_for_object_head.edge_index_supervision[0], graph_data_for_object_head.edge_index_negative[0]))
    top10_heads = head_objects[sorted_by_scores_for_object_head_indcs[:10]]
    h_pred_top10.append(top10_tails)

    # print('')
    ############################################################################## 1
    # if False:
    #   hit_index = (
    #             tail_objects[sorted_by_scores_for_object_head_indcs[:500]] == eval_triplet[2]
    #           ).nonzero(as_tuple=True)[0] + 1
    #   if hit_index.shape[0]:
    #     print(f'Tail ranked at {hit_index.item()}')
    #     print('-' * 30)
    # ############################################################################### 2

    # ############################################################################### 3
    # hit_index = (
    #             head_objects[sorted_by_scores_for_object_tail_indcs[:500]] == eval_triplet[0]
    #           ).nonzero(as_tuple=True)[0] + 1
    # if hit_index.shape[0]:
    #   print(f'Head ranked at {hit_index.item()}')
    #   print('-' * 30)
    ############################################################################## 4
    
  model.train()
  return torch.stack(h_pred_top10), torch.stack(t_pred_top10)

In [60]:
#@title evaluate_hits_at_10(model, mode) FUNCTION{ form-width: "15%" }
def evaluate_hits_at_10(model: GNN, mode:str) -> torch.float:
  if mode == 'validation':
    head_objects_top10, tail_objects_top10 = evaluation_rank(model, val_set, train_set)
    return evaluator.eval(
            head_objects_top10,
            tail_objects_top10,
            val_set,
            False
          )
  elif mode == 'testing':
    head_objects_top10, tail_objects_top10 = evaluation_rank(model, test_set, torch.cat((train_set, val_set), dim=0))
    return evaluator.eval(
            head_objects_top10,
            tail_objects_top10,
            test_set,
            False
          )


In [61]:
#@title save_execution() FUNCTION { form-width: "15%" }
def save_execution(model, iteration, epoch, epoch_loss):
  saved_time = str(datetime.now())
  torch.save(model, f'model@{saved_time}.pth')
  with open(f'info@{saved_time}.txt', 'w') as info_file:
    info_file.write(f'model: {str(model)}')
    info_file.write('\n')
    info_file.write(f'iteration: {iteration}\n')
    info_file.write(f'epoch: {epoch + 1}\n')
    info_file.write(f'epoch loss: {epoch_loss}')

  # from google.colab import drive
  # drive.mount('/content/drive')
  # import shutil
  # shutil.copy(f'/content/model@{saved_time}.pth', f'/content/drive/model@{saved_time}.pth')
  # shutil.copy(f'/content/info@{saved_time}.txt', f'/content/drive/info@{saved_time}.txt')

#Trainer

In [ ]:
# @title Parameter and Hyperparameter Tuning MAIN{ form-width: "10%" }
shallow_model.reset_parameters()
try:
  for epoch in range(5):
    iteration = 0
    epoch_loss = 0
  ############################## TRAIN Graph Maker #####################################
    for i in tqdm(range(0, train_set.shape[0] - Global.MINI_BATCH_SIZE.value, Global.MINI_BATCH_SIZE.value)):
      iteration += 1
      train_supervision = train_set[i: i + Global.MINI_BATCH_SIZE.value, :]
      train_messaging = torch.cat(
          (train_set[: i, :], train_set[i + Global.MINI_BATCH_SIZE.value: , :]),
          dim=0
      )
      train_graph = mini_batch_maker(
          train_messaging,
          train_supervision,
          features,
          'one-hot'
      )
      loss, positive_score, negative_score = train(shallow_model, train_graph, opt_shallow, margin_ranking_loss)
      epoch_loss += loss
      if iteration % 250 == 0:
        print('')
        print('-' * 50)
        agg_p_score = positive_score.sum().item()
        agg_n_score = negative_score.sum().item()
        print(f'Train Batch {iteration}:')
        print('-' * 50)
        print(f'Batch Loss:    {loss: .4f}')
        print('-' * 50)
        print(f'Average Loss:         <{epoch_loss / iteration: .4f} >')
        print('-' * 50)
        print(f'Agg P-Score:   {agg_p_score: .4f}')
        print('-' * 50)
        print(f'Agg N-Score:   {agg_n_score: .4f}')
        print(f'===' * 25)

        ###########################################
        # a, b = evaluation_rank(model, val_set[1000:1100], train_set)
        # evaluator.eval(
        #             a,
        #             b,
        #             val_set[1000:1100],
        #             False
        #           )
        # print(f'***' * 25)
        ###########################################
      # if iteration % 1500 == 0:
      #   save_execution(model_shallow, iteration, epoch, epoch_loss)

    evaluate_hits_at_10(model_shallow, mode='validation')
    
except KeyboardInterrupt:
  # save_execution(model, iteration, epoch, epoch_loss)
  pass


# TEST EVALUATION

In [ ]:
#@title
a, b = evaluation_rank(shallow_model, val_set[1000:1100], train_set)
evaluator.eval(
            a,
            b,
            val_set[1000:1100],
            False
          )

# (h, r, t) (h, r, t')

In [ ]:
#@title
try:
  evaluate_hits_at_10(model, 'testing')
except KeyboardInterrupt:
  pass

# ADD-ONS


In [ ]:
#@title add_tail_to_head(data_split) FUNCTION { form-width: "15%" }
def add_tail_to_head(data_split: torch.Tensor, plus: int=28) -> torch.Tensor:
  heads = data_split[:, (0)]
  tails = data_split[:, (-1)]
  relations = data_split[:, (1)]
  
  tail_to_head = torch.vstack(
      (tails, relations + plus, heads)
  ).t()

  return torch.cat(
      (data_split, tail_to_head),
      dim=0
  )



In [ ]:
#@title checksum(data_subset, graph_data) FUNCTION { form-width: "15%" }
def checksum(data_subset: torch.Tensor, graph_data: Data, step: int=500) -> NoReturn:
  for i in range(step, data_subset.shape[0], step):
    graph_head_indcs = graph_data.edge_index[0, i-step:i]
    data_head_entities = data_subset[i-step:i, 0]
    graph_head_entities = graph_data.x[graph_head_indcs].reshape(-1)

    checksum_head = (graph_head_entities == data_head_entities).sum().item()
    if not checksum_head == step:
      print('head')
      print(i)
      break

    graph_relation_indcs = graph_data.edge_type[i-step:i]
    data_relation_types = data_subset[i-step:i, 1]

    checksum_relation = (graph_relation_indcs == data_relation_types).sum().item()
    if not checksum_relation == step:
      print('relation')
      print(i)
      break

    graph_tail_indcs = graph_data.edge_index[1, i-step:i]
    data_tail_entities = data_subset[i-step:i, 2]
    graph_tail_entities = graph_data.x[graph_tail_indcs].reshape(-1)

    checksum_tail = (graph_tail_entities == data_tail_entities).sum().item()
    if not checksum_head == step:
      print('tail')
      print(i)
      break

  else:
    print('All clear :)')

In [ ]:
#@title partial_graph_data_maker(dataset, x_feature) FUNCTION{ form-width: "15%" }
def partial_graph_data_maker(messaging: torch.Tensor, supervision: torch.Tensor, negative_samples: torch.Tensor, x_feature: str=Global.FEATURE_ENG.value, check_for_correctness: bool=False) -> Data:
  relation_idx = Global.RELATION_INDEX.value
  head_idx = Global.HEAD_INDEX.value
  tail_idx = Global.TAIL_INDEX.value


  if x_feature == 'one-hot':
    one_hot_index = 0
    edge_index_list_flat = []
    seen_dict = defaultdict(lambda: -1)
    edge_index_flat = messaging[:, (head_idx, tail_idx)].reshape(-1)

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_messaging = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    edge_index_flat = supervision[:, (head_idx, tail_idx)].reshape(-1)
    edge_index_list_flat = []

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_supervision = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    edge_index_flat = negative_samples[:, (head_idx, tail_idx)].reshape(-1)
    edge_index_list_flat = []

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_negative = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    x = torch.tensor(list(seen_dict.keys())).reshape(-1, 1)

    graph_data = Data(
        x=x,
        edge_index_messaging=edge_index_messaging.t().contiguous(),
        edge_type_messaging=messaging[:, relation_idx],
        edge_index_supervision=edge_index_supervision.t().contiguous(),
        edge_type_supervision=supervision[:, relation_idx],
        edge_index_negative=edge_index_negative.t().contiguous(),
        edge_type_negative=negative_samples[:, (relation_idx)]
    )
  else:
    raise NotImplementedError('This functionality has not been implemented yet.')

  if check_for_correctness:
    checksum(dataset, graph_data)
  return graph_data

In [ ]:
#@title visualize_graph(graph_data, height, width) FUNCTION{ form-width: "15%" }
def visualize_graph(graph_data: Data, height: int=10, width:int=10) -> NoReturn:
  nx_graph = to_networkx(graph_data)

  pos = nx.spring_layout(nx_graph)

  fig = plt.gcf()
  fig.set_size_inches(width, height)

  edge_labels = dict()
  ei_np = graph_data.edge_index.t().numpy()
  for edge in nx_graph.edges():
    e = np.array(edge)
    idx = np.where(ei_np == e)[0][0]
    label = graph_data.edge_type[idx].item()
    edge_labels.update({edge: label})
    
  nx.draw_networkx_nodes(nx_graph, pos)
  nx.draw_networkx_edges(nx_graph, pos, connectionstyle='arc3,rad=0.2')
  nx.draw_networkx_labels(nx_graph, pos, labels={n:graph_data.x[n].item() for n in nx_graph})
  nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels)

  fig.show()


In [ ]:
#@title Visualization OPTIONAL-MAIN { form-width: "15%" }
data_subset = train_set[:2500, :]
graph_data = graph_data_maker(data_subset, 'one-hot')
visualize_graph(graph_data, 100, 100)

#Extra

In [62]:
#@title DM
class DistMult(Module):
  def __init__(self, em_size, dropout_r=0.5):
    super(DistMult, self).__init__()
    self.em_size = em_size
    self.entity_encoder = Embedding(features.shape[0], em_size)
    self.relation_encoder = Embedding(Global.NUM_RELATIONS.value, em_size)
    self.p_entity_bn = torch.nn.BatchNorm1d(em_size)
    self.n_entity_bn = torch.nn.BatchNorm1d(em_size)
    self.relation_bn = torch.nn.BatchNorm1d(em_size)
    # self.dropout = torch.nn.Dropout(dropout_r)
  
  def forward(self, x_supervision, x_contrast):
    sup_heads = self.entity_encoder(x_supervision[:, 0])
    sup_heads = self.p_entity_bn(sup_heads)
    # p_heads = self.dropout(p_heads)

    sup_relations = self.relation_encoder(x_supervision[:, 1])
    sup_relations = self.relation_bn(sup_relations)
    # p_relations = self.dropout(p_relations)

    sup_tails = self.entity_encoder(x_supervision[:, 2])
    sup_tails = self.p_entity_bn(sup_tails)
    # p_tails = self.dropout(p_tails)

    con_heads = self.entity_encoder(x_contrast[:, 0])
    con_heads = self.n_entity_bn(con_heads)
    # n_heads = self.dropout(n_heads)

    con_relations = self.relation_encoder(x_contrast[:, 1])
    con_relations = self.relation_bn(con_relations)
    # n_relations = self.dropout(n_relations)

    con_tails = self.entity_encoder(x_contrast[:, 2])
    con_tails = self.n_entity_bn(con_tails)
    # n_tails = self.dropout(n_tails)

    sup_scores = (sup_heads * sup_relations * sup_tails).sum(1)
    con_scores = (con_heads * con_relations * con_tails).sum(1)
    return sup_scores, con_scores


In [ ]:
#@title RSC
class RESCAL(Module):
  def __init__(self, em_size, dropout_r=0.5):
    super(RESCAL, self).__init__()
    self.em_size = em_size
    self.entity_encoder = Embedding(features.shape[0], em_size)
    self.relation_encoder = Embedding(Global.NUM_RELATIONS.value, em_size ** 2)
    self.entity_bn = torch.nn.BatchNorm1d(em_size)
    self.relation_bn = torch.nn.BatchNorm1d(em_size ** 2)
    self.dropout = torch.nn.Dropout(dropout_r)


  def score(self, heads, relations, tails):
    tails = tails.view(-1, self.em_size, 1)
    relations = relations.view(-1, self.em_size, self.em_size)
    tr = torch.matmul(relations, tails).view(-1, self.em_size)
    return torch.sum(heads * tr, -1)


  def forward(self, x_p, x_n):
    p_heads = self.entity_encoder(x_p[:, 0])
    p_heads = self.entity_bn(p_heads)
    p_heads = self.dropout(p_heads)

    p_relations = self.relation_encoder(x_p[:, 1])
    p_relations = self.relation_bn(p_relations)
    p_relations = self.dropout(p_relations)

    p_tails = self.entity_encoder(x_p[:, 2])
    p_tails = self.entity_bn(p_tails)
    p_tails = self.dropout(p_tails)

    n_heads = self.entity_encoder(x_n[:, 0])
    n_heads = self.entity_bn(n_heads)
    n_heads = self.dropout(n_heads)

    n_relations = self.relation_encoder(x_contrast[:, 1])
    n_relations = self.relation_bn(n_relations)
    n_relations = self.dropout(n_relations)

    n_tails = self.entity_encoder(x_contrast[:, 2])
    n_tails = self.entity_bn(n_tails)
    n_tails = self.dropout(n_tails)

    p_scores = self.score(p_heads, p_relations, p_tails)
    n_scores = self.score(n_heads, n_relations, n_tails)
    return p_scores, n_scores


In [63]:
#@title NS
def ns(supervision, candidates):
  heads = supervision[:, 0]
  relations = supervision[:, 1]
  tails = supervision[:, -1]

  # ct_size = supervision.shape[0] // 2
  # ch_size = supervision.shape[0] - ct_size
  negative_samples_corrupted_tails = torch.vstack(
      (
          heads, 
          relations, 
          torch.multinomial(
              candidates.type(torch.float).to(Global.DEVICE.value), 
              heads.shape[0]
          )
      )
  ).t().contiguous()

  negative_samples_corrupted_heads = torch.vstack(
      ( 
          torch.multinomial(
              candidates.type(torch.float).to(Global.DEVICE.value), 
              tails.shape[0]
          ), 
          relations, 
          tails
      )
  ).t().contiguous()

  negative_samples = torch.cat(
      (negative_samples_corrupted_heads, negative_samples_corrupted_tails),
      dim=0
  )
  return negative_samples

In [64]:
#@title Models
dm = DistMult(384).to(Global.DEVICE.value)
# rsc = RESCAL(256, 0.2).to(Global.DEVICE.value)
lf = torch.nn.MarginRankingLoss(margin=1, reduction='mean')
alpha = 1e-3
decay_rate = 1e-7
opt_dm = torch.optim.Adam(dm.parameters(), lr=alpha, weight_decay=decay_rate)
# opt_rsc = torch.optim.Adam(rsc.parameters(), lr=alpha)
print(dm)
batch_size = 1024
# print(rsc)


DistMult(
  (entity_encoder): Embedding(180992, 384)
  (relation_encoder): Embedding(28, 384)
  (p_entity_bn): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (n_entity_bn): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relation_bn): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [65]:
#@title eval { form-width: "15%" }
@torch.no_grad()
def eval(model, eval_set, use_tqdm=True):
  h_pred_top10 = list()
  t_pred_top10 = list()

  if use_tqdm:
    fn = tqdm
  else:
    fn = lambda z: z

  for eval_triplet in fn(eval_set):
    #use tqdm
    psuedo_negative_triplets_corrupted_head, psuedo_negative_triplets_corrupted_tail = get_psuedo_negative_triplets(eval_triplet)
    

    model.eval()
    scores_for_object_tail = torch.cat(model(eval_triplet.reshape(1, 3), psuedo_negative_triplets_corrupted_tail))


    sorted_by_scores_for_object_tail_indcs = torch.sort(scores_for_object_tail, descending=True)[1]

    # print('')
    # print(torch.tensor([eval_triplet[0]]).shape)

    tail_objects = torch.cat((torch.tensor([eval_triplet[2]]).to(Global.DEVICE.value), psuedo_negative_triplets_corrupted_tail[:, 2]))
    top10_tails = tail_objects[sorted_by_scores_for_object_tail_indcs[:10]]
    t_pred_top10.append(top10_tails)

    model.eval()
    scores_for_object_head = torch.cat(model(eval_triplet.reshape(1, 3), psuedo_negative_triplets_corrupted_head))

    sorted_by_scores_for_object_head_indcs = torch.sort(scores_for_object_head, descending=True)[1]

    head_objects = torch.cat((torch.tensor([eval_triplet[0]]).to(Global.DEVICE.value), psuedo_negative_triplets_corrupted_head[:, 0]))
    top10_heads = head_objects[sorted_by_scores_for_object_head_indcs[:10]]
    h_pred_top10.append(top10_heads)

    
  model.train()
  return torch.stack(h_pred_top10), torch.stack(t_pred_top10)

In [66]:
#@title hits at 10 { form-width: "15%" }
def hits_at10(evaluator, h_pred_top10, t_pred_top10, triples):
    """
    Evaluates ranked lists of head and tail entity predictions for a set of evaluation triples. By default creates a submission file.

    Args:
        h_pred_top10: Top 10 predictions for the head entity. The value at (i,j) is the ID of the predicted head entity with rank `j+1` for the triple `triples[i]`. Shape `(num_eval_triplets,10)`
        t_pred_top10: Top 10 predictions for the tail entity. The value at (i,j) is the ID of the predicted tail entity with rank `j+1` for the triple `triples[i]`. Shape `(num_eval_triplets,10)`
        triples: Set of evaluation triples. Shape `(num_eval_triplets,3)`
        # save_submission: If true a submission file is created. Default: True
    """

    assert t_pred_top10.shape[1] == h_pred_top10.shape[1] == 10 and t_pred_top10.shape[0] == h_pred_top10.shape[
        0] == triples.shape[0]

    # h,r->t
    t_pred_top10 = evaluator._to_torch(t_pred_top10)
    t_correct_index = evaluator._to_torch(triples[:, 2])
    h_pred_top10 = evaluator._to_torch(h_pred_top10)
    h_correct_index = evaluator._to_torch(triples[:, 0])
    pred_top10 = torch.cat((t_pred_top10, h_pred_top10), dim=0)
    correct_index = torch.cat((t_correct_index, h_correct_index), dim=0)

    hits10 = evaluator._calculate_h10(correct_index.to(pred_top10.device), pred_top10)
    # if save_submission is True:
    #     self._save_test_submission(pred_top10)

    # print("Please copy also the following line in the respective field of the submission form:")
    # print({'h10': h10})
    return hits10

In [ ]:
#@title MRR eval
@torch.no_grad()
def mrr(eval_set: torch.Tensor):
  r = 0
  for triple in tqdm(eval_set):
    ch, ct = get_psuedo_negative_triplets(triple)
    negative = torch.cat((ch, ct), dim=0)
    dm.eval()
    p, n = dm(triple.reshape(-1, 3), negative)
    # top10 = torch.topk(n, 10)[0]
    rank = (p < n).sum() + 1
    # print(rank)
    r += (1 / rank)
    # print(rank.item() if rank < 11 else 'not in top 10')
    # print('=====================' * 3)
  return (r/eval_set.shape[0]).item()
  

In [67]:
#@title loss { form-width: "15%" }
def calc_margin_based_ranking_loss(p_score, n_score, lf):
  n = n_score.repeat(p_score.shape)
  p = p_score.repeat((n_score.shape[0], 1)).t().reshape(-1)
  return lf(p, n, torch.ones(p_score.shape[0] * n_score.shape[0]).to(Global.DEVICE.value))

In [ ]:
#@title trainer { form-width: "15%" }
try:
  for epoch in range(1, 3):
    # a = 0
    epoch_loss = 0
    iteration = 0
    for i in tqdm(range(0, train_set.shape[0]-batch_size, batch_size)):
      opt_dm.zero_grad()
      # a = 0
      # aa = 0
      iteration += 1
      supervision = train_set[i: i+batch_size, :]
      negative = ns(supervision, features)
      dm.train()
      p_score, n_score = dm(supervision, negative)
      loss = calc_margin_based_ranking_loss(p_score, n_score, lf)
      loss.backward()
      epoch_loss += loss.item()
      opt_dm.step()
      if iteration % 100 == 0:
        indcs = torch.randperm(val_set.shape[0])[:100]
        h, t = eval(dm, val_set[indcs], False)
        h10 = hits_at10(evaluator, h, t, val_set[indcs])
        print('')
        print('*' * 32)
        print(f'Batch Loss: {loss.item()}')
        # print('*' * 40)
        print(f'Hits @ 10: {h10}')
        print('*' * 32)
    print('='*40)
    print(f'Epoch {epoch}, Loss: {epoch_loss / iteration}')
    print('='*40)
    if epoch % 10 == 0:
      h, t = eval(dm, val_set[:12000])
      print('*' * 40)
      evaluator.eval(
          h,
          t,
          val_set[:12000],
          False
      )
      print('*' * 40)
except KeyboardInterrupt:
  save_execution(dm, iteration, epoch, epoch_loss)

  0%|          | 3/4093 [00:06<2:10:58,  1.92s/it]

In [ ]:
#@title
h, t = eval(dm, test_set[90000:])
evaluator.eval(
  h,
  t,
  test_set[90000:],
  False
)

In [ ]:
with torch.no_grad():
  dm.eval()
  p = test_set[:5]
  n = torch.randint(1, 20, (5, 3)).to(Global.DEVICE.value)
  n_score, p_score = dm(n, p)
  pr_score, nr_score = dm(p, n)
  print(p_score) 
  print(pr_score)
  print(n_score) 
  print(nr_score)

tensor([-10.9281, -17.1026,   5.1132,  -0.1364, -14.1778])
tensor([ 12.9912, -24.9903,   2.9273,  -4.5373,  19.3820])
